<a href="https://colab.research.google.com/github/asad-777/Piaic_Gen-Ai/blob/main/Video_Audio_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q -U google-genai

In [47]:
!pip install -q -U gTTS

In [6]:
!pip install -q -U playsound

  Preparing metadata (setup.py) ... done


In [2]:
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')

In [42]:
from google import genai
from google.genai import Client
import time
from google.genai.types import Content, Part

In [24]:
client = genai.Client(
    api_key=api_key
)

In [25]:
model = "gemini-2.0-flash-exp"

In [26]:
video_file_name = "/content/videoplayback.mp4"

# upload your video here, i uploaded a youtube video for example

In [29]:
def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
    print("Please Wait !")
    time.sleep(5)
    video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print("Video processing complete", (video_file.uri or ""))

  return video_file

my_vid = upload_video(video_file_name)

Please Wait !
Video processing complete https://generativelanguage.googleapis.com/v1beta/files/xf6zmvgmvcu8


In [51]:
prompt = "For each scene in this video, generate a caption for it,dont summarize anything, just captions"

In [37]:
video = my_vid

In [52]:
response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role = "user",
            parts = [
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or "",

                )
            ]
        ),
        prompt,
    ]
)

print(response.text)

Certainly, here are captions for each scene in the video:

[00:00:00-00:00:22] A man with dark curly hair, wearing a yellow T-shirt, looks into the camera while speaking.


In [49]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

In [54]:
tts = gTTS(text=response.text, lang='en')

with open("output.mp3", "wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3", autoplay=True))